In [1]:
import numpy as np
import sklearn.datasets as sk_dataset
from sklearn.model_selection import train_test_split, KFold
from scipy.io import loadmat

In [28]:
n_node = 10 # num of nodes in hidden layer
lam = 1 # regularization parameter, lambda
weight_range = [-1, 1] # range of random weights
bias_range = [0, 1] # range of random biases

class RVFL:
    """ RVFL Classifier """
    
    def __init__(self, n_node, lam, w_range, b_range, activation='relu', same_feature=False):
        self.n_node = n_node
        self.lam = lam
        self.w_range = w_range
        self.b_range = b_range
        self.weight = None
        self.bias = None
        self.beta = None
        a = Activation()
        self.activation_function = getattr(a, activation)
        self.std = None
        self.mean = None
        self.same_feature = same_feature
        
    def train(self, data, label, n_class):
        assert len(data.shape) > 1
        assert len(data) == len(label)
        assert len(label.shape) == 1
        
        data = self.standardize(data) # Normalize
        n_sample = len(data)
        n_feature = len(data[0])
        self.weight = (self.w_range[1] - self.w_range[0]) * np.random.random([n_feature, self.n_node]) + self.w_range[0]
        self.bias = (self.b_range[1] - self.b_range[0]) * np.random.random([1, self.n_node]) + self.b_range[0]
        
        h = self.activation_function(np.dot(data, self.weight) + np.dot(np.ones([n_sample, 1]), self.bias))
        d = np.concatenate([h, data], axis=1)
        d = np.concatenate([d, np.ones_like(d[:, 0:1])], axis=1) # concat column of 1s
        y = self.one_hot_encoding(label, n_class)
        # Minimize training complexity
        if n_sample > (self.n_node + n_feature):
            self.beta = np.linalg.inv((self.lam * np.identity(d.shape[1]) + np.dot(d.T, d))).dot(d.T).dot(y)
        else:
            self.beta = d.T.dot(np.linalg.inv(self.lam * np.identity(n_sample) + np.dot(d, d.T))).dot(y)
            
    def predict(self, data, raw_output=False):
        data = self.standardize(data) # Normalize
        h = self.activation_function(np.dot(data, self.weight) + self.bias)
        d = np.concatenate([h, data], axis=1)
        d = np.concatenate([d, np.ones_like(d[:, 0:1])], axis=1)
        result = self.softmax(np.dot(d, self.beta))
        if not raw_output:
            result = np.argmax(result, axis=1)
        return result
    
    def eval(self, data, label):
        assert len(data.shape) > 1
        assert len(data) == len(label)
        assert len(label.shape) == 1
        
        data = self.standardize(data)  # Normalize
        h = self.activation_function(np.dot(data, self.weight) + self.bias)
        d = np.concatenate([h, data], axis=1)
        d = np.concatenate([d, np.ones_like(d[:, 0:1])], axis=1)
        result = np.dot(d, self.beta)
        result = np.argmax(result, axis=1)
        acc = np.sum(np.equal(result, label))/len(label)
        return acc
        
    def one_hot_encoding(self, label, n_class):
        y = np.zeros([len(label), n_class])
        for i in range(len(label)):
            y[i, label[i]] = 1
        return y
    
    def standardize(self, x):
        if self.same_feature is True:
            if self.std is None:
                self.std = np.maximum(np.std(x), 1/np.sqrt(len(x)))
            if self.mean is None:
                self.mean = np.mean(x)
            return (x - self.mean) / self.std
        else:
            if self.std is None:
                self.std = np.maximum(np.std(x, axis=0), 1/np.sqrt(len(x)))
            if self.mean is None:
                self.mean = np.mean(x, axis=0)
            return (x - self.mean) / self.std
        
    def softmax(self, x):
        return np.exp(x) / np.repeat((np.sum(np.exp(x), axis=1))[:, np.newaxis], len(x[0]), axis=1)
        
class Activation:
    def sigmoid(self, x):
        return 1 / (1 + np.e ** (-x))
    
    def sine(self, x):
        return np.sin(x)
    
    def sign(self, x):
        return np.sign(x)
    
    def relu(self, x):
        return np.maximum(0, x)


In [3]:
# coil20 dataset
if __name__=="__main__":
    dataset = loadmat('coil20.mat')
    label = np.array([dataset['Y'][i][0] - 1 for i in range(len(dataset['Y']))])
    data = dataset['X']
    n_class = len(np.unique(label))

    # train-test-split
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)
    kf = KFold(10, True, 1)
    val_acc = []
    max_index = -1
    
    for i, kf_values in enumerate(kf.split(X_train, y_train)):
    #     print(f'train: {train_index}, val: {val_index}')
        print('Validation: {}'.format(i + 1))
        train_index, val_index = kf_values
        X_val_train, X_val_test = X_train[train_index], X_train[val_index]
        y_val_train, y_val_test = y_train[train_index], y_train[val_index]
        rvfl = RVFL(n_node, lam, weight_range, bias_range, 'relu', False)
        rvfl.train(X_val_train, y_val_train, n_class)
        prediction = rvfl.predict(X_val_test, True)
        acc = rvfl.eval(X_val_test, y_val_test)
        print(f'Validation accuracy: {acc}')
        val_acc.append(acc)
        if acc >= max(val_acc):
            max_index = train_index

    X_train, y_train = X_train[max_index], y_train[max_index]
    rvfl = RVFL(n_node, lam, weight_range, bias_range, 'relu', False)
    rvfl.train(X_train, y_train, n_class)
    prediction = rvfl.predict(X_test, True)
    acc = rvfl.eval(X_test, y_test)
    print(f'Accuracy: {acc}')

c:\users\yc000\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Validation: 1
Validation accuracy: 0.9482758620689655
Validation: 2
Validation accuracy: 0.9568965517241379
Validation: 3
Validation accuracy: 0.9304347826086956
Validation: 4
Validation accuracy: 0.9478260869565217
Validation: 5
Validation accuracy: 0.9043478260869565
Validation: 6
Validation accuracy: 0.9652173913043478
Validation: 7
Validation accuracy: 0.9739130434782609
Validation: 8
Validation accuracy: 0.9652173913043478
Validation: 9
Validation accuracy: 0.9478260869565217
Validation: 10
Validation accuracy: 0.9043478260869565
Accuracy: 0.9375


In [29]:
# g50c dataset
if __name__=="__main__":
    dataset = loadmat('g50c.mat')
    label = np.array([dataset['y'][i][0] for i in range(len(dataset['y']))])
    data = dataset['X']
    n_class = len(np.unique(label))
    
    # train-test-split
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)
    kf = KFold(10, True, 1)
    val_acc = []
    max_index = -1
    
    for i, kf_values in enumerate(kf.split(X_train, y_train)):
    #     print(f'train: {train_index}, val: {val_index}')
        print('Validation: {}'.format(i + 1))
        train_index, val_index = kf_values
        X_val_train, X_val_test = X_train[train_index], X_train[val_index]
        y_val_train, y_val_test = y_train[train_index], y_train[val_index]
        rvfl = RVFL(n_node, lam, weight_range, bias_range, 'relu', False)
        rvfl.train(X_val_train, y_val_train, n_class)
        prediction = rvfl.predict(X_val_test, True)
        acc = rvfl.eval(X_val_test, y_val_test)
        print(f'Validation accuracy: {acc}')
        val_acc.append(acc)
        if acc >= max(val_acc):
            max_index = train_index

    X_train, y_train = X_train[max_index], y_train[max_index]
    rvfl = RVFL(n_node, lam, weight_range, bias_range, 'relu', False)
    rvfl.train(X_train, y_train, n_class)
    prediction = rvfl.predict(X_test, True)
    acc = rvfl.eval(X_test, y_test)
    print(f'Accuracy: {acc}')

Validation: 1
Validation accuracy: 0.4318181818181818
Validation: 2
Validation accuracy: 0.4772727272727273
Validation: 3
Validation accuracy: 0.5454545454545454
Validation: 4
Validation accuracy: 0.5
Validation: 5
Validation accuracy: 0.29545454545454547
Validation: 6
Validation accuracy: 0.5909090909090909
Validation: 7
Validation accuracy: 0.4318181818181818
Validation: 8
Validation accuracy: 0.5227272727272727
Validation: 9
Validation accuracy: 0.5454545454545454
Validation: 10
Validation accuracy: 0.5681818181818182
Accuracy: 0.5363636363636364


In [22]:
# uspst dataset
if __name__=="__main__":
    dataset = loadmat('uspst.mat')
    label = np.array([dataset['y'][i][0] for i in range(len(dataset['y']))])
    data = dataset['X']
    n_class = len(np.unique(label))

    # train-test-split
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)
    kf = KFold(10, True, 1)
    val_acc = []
    max_index = -1
    
    for i, kf_values in enumerate(kf.split(X_train, y_train)):
    #     print(f'train: {train_index}, val: {val_index}')
        print('Validation: {}'.format(i + 1))
        train_index, val_index = kf_values
        X_val_train, X_val_test = X_train[train_index], X_train[val_index]
        y_val_train, y_val_test = y_train[train_index], y_train[val_index]
        rvfl = RVFL(n_node, lam, weight_range, bias_range, 'relu', False)
        rvfl.train(X_val_train, y_val_train, n_class)
        prediction = rvfl.predict(X_val_test, True)
        acc = rvfl.eval(X_val_test, y_val_test)
        print(f'Validation accuracy: {acc}')
        val_acc.append(acc)
        if acc >= max(val_acc):
            max_index = train_index

    X_train, y_train = X_train[max_index], y_train[max_index]
    rvfl = RVFL(n_node, lam, weight_range, bias_range, 'relu', False)
    rvfl.train(X_train, y_train, n_class)
    prediction = rvfl.predict(X_test, True)
    acc = rvfl.eval(X_test, y_test)
    print(f'Accuracy: {acc}')

Validation: 1
Validation accuracy: 0.8633540372670807
Validation: 2
Validation accuracy: 0.8509316770186336
Validation: 3
Validation accuracy: 0.8757763975155279
Validation: 4
Validation accuracy: 0.8944099378881988
Validation: 5
Validation accuracy: 0.9130434782608695
Validation: 6
Validation accuracy: 0.90625
Validation: 7
Validation accuracy: 0.81875
Validation: 8
Validation accuracy: 0.89375
Validation: 9
Validation accuracy: 0.91875
Validation: 10
Validation accuracy: 0.9125
Accuracy: 0.8955223880597015
